In [38]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm

# jaccard文本相似度
def get_jaccard_sim(str1, str2):
    a = set(str1.split()) 
    b = set(str2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

def add_space(x):
    if (' ' not in x)&(len(x)<=5):
        return ' ' + x + ' '
    else:
        return x

def product_name_postprocess(x):
    x = str(x)
    x = x.replace('-',' ')
    x = x.strip()
    x = add_space(x)
    return x

def product_name_postprocessV2(x):
    x = str(x)
    x = x.replace('-','')
    x = x.strip()
    return x

def Collection_method(df,產品集合,x_col):
    labels = {}
    labels_max = {}
    for i in tqdm(df.index):
        products = []
        for p in 產品集合:
            if (str(p) in str(df.loc[i,x_col])) | (get_jaccard_sim(str(p),str(df.loc[i,x_col]))>=0.9): # 模糊比對
                products.append(str(p)) # 加入候選清單
        if len(products) > 0: # 如果有找到產品 
            labels[i] = products # 複數個產品,之後配合公司去篩選出一個
            labels_max[i] = max(products,key=len) # 取長度最長的產品
        else:
            labels[i] = 'not find'
            labels_max[i] = 'not find'
    predict = pd.DataFrame(index=labels.keys(),columns=['預測產品'])
    predict['預測產品'] = labels.values()
    predict['預測產品(取長度最長)'] = labels_max.values()
    predict['預測產品使用方式'] = 'rule'
    return predict

# 讀取訓練資料(SPEC)
train_df = pd.read_csv('../data/preprocess_for_SQUAD_產品.csv')[['string_X_train','Y_label','EXPNO']]
train_df_不加空白版本 = train_df.copy()#
train_df['Y_label'] = train_df['Y_label'].apply(lambda x:product_name_postprocess(x))
train_df_不加空白版本['Y_label'] = train_df_不加空白版本['Y_label'].apply(lambda x:product_name_postprocessV2(x)) #品名後處理

# 讀取台塑網提供之(寶典人工手動修正過刪除線問題)
root = '../data/寶典/寶典人工處理後/'

df5 = pd.read_excel(root+'寶典.v6.20211020.xlsx',engine='openpyxl')[['CODIV','DIVNM','ITEMNM']]
df5 = df5.rename(columns={'ITEMNM':'品名','DIVNM':'公司事業部門','CODIV':'公司代號'})

# 我做的寶典
df_by_ricky = pd.read_excel(root+'寶典_by_ricky.xlsx',engine='openpyxl')[['CODIV','DIVNM','ITEMNM']]
df_by_ricky = df_by_ricky.rename(columns={'ITEMNM':'品名','DIVNM':'公司事業部門','CODIV':'公司代號'})

#df = df1.append(df2).append(df3).append(df4).append(df5) # 合併所有寶典
df = df5.append(df_by_ricky) # 合併官方寶典和我做的寶典
df_不加空白版本 = df.copy()
df['品名'] = df['品名'].apply(lambda x:product_name_postprocess(x)) #品名後處理
df_不加空白版本['品名'] = df_不加空白版本['品名'].apply(lambda x:product_name_postprocessV2(x)) #品名後處理

# 製作對應表(訓練資料對代號)
品名2代號訓練資料 = dict(zip(train_df.dropna(subset=['EXPNO'],axis=0)['Y_label'],train_df.dropna(subset=['EXPNO'],axis=0)['EXPNO']))

# 製作產品集合(寶典+SPEC)
產品集合 = set(df['品名'].values.tolist())# + train_df['Y_label'].values.tolist())
產品集合_不加空白版本 = set(df_不加空白版本['品名'].values.tolist() + train_df_不加空白版本['Y_label'].values.tolist())

# 製作對應表(寶典對部門和代號)
品名2部門寶典 = dict(zip(df['品名'],df['公司事業部門']))
品名2代號寶典 = dict(zip(df['品名'],df['公司代號']))

def map2部門(x):
    if x in 品名2部門寶典.keys(): #先從寶典找
        return str(品名2部門寶典[x])
    elif x in train_df['Y_label'].values.tolist(): #找不到從訓練資料找
        return str(find_department(x))
    else:# 模糊比對
        print('模糊比對')
        jacs = {}
        for i in 品名2部門寶典.keys():
            jacs[i] = get_jaccard_sim(x,i)
        x = max(jacs,key=jacs.get) # 模糊比對
        return map2部門(x)

def map2代號(x):
    if  x in 品名2代號寶典.keys(): #先從寶典找
        return str(品名2代號寶典[x])
    elif x in 品名2代號訓練資料.keys(): #找不到從訓練資料找
        return str(品名2代號訓練資料[x])
    else:# 模糊比對
        print('模糊比對')
        jacs = {}
        for i in 品名2代號寶典.keys():
            jacs[i] = get_jaccard_sim(x,i)
        x = max(jacs,key=jacs.get) # 模糊比對
        return map2代號(x)

NAME = "ABS RESIN"
test_df = pd.DataFrame()
test_df['45A'] = [NAME]
test_df

,45A
0,ABS RESIN


In [39]:
res = Collection_method(test_df,產品集合,'45A')
res['部門'] = map2部門(res['預測產品(取長度最長)'].values[0])
res['代號'] = map2代號(res['預測產品(取長度最長)'].values[0])
res

C:\Users\ricky\AppData\Local\Temp/ipykernel_10536/4058755502.py:33: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(df.index):


,預測產品,預測產品(取長度最長),預測產品使用方式,部門,代號
0,"(ABS RESIN,)",ABS RESIN,rule,台化塑膠事業部,4A


In [40]:
res = Collection_method(test_df,產品集合_不加空白版本,'45A')
res['部門'] = map2部門(res['預測產品(取長度最長)'].values[0])
res['代號'] = map2代號(res['預測產品(取長度最長)'].values[0])
res

C:\Users\ricky\AppData\Local\Temp/ipykernel_10536/4058755502.py:33: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(df.index):


,預測產品,預測產品(取長度最長),預測產品使用方式,部門,代號
0,"(RESIN, ABS RESIN, ABS)",ABS RESIN,rule,台化塑膠事業部,4A
